In [1]:
import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import cv2

# from tensorflow.keras.applications import Densnet
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.data import Dataset

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from itertools import product
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skmultilearn.model_selection import iterative_train_test_split
from wandb.keras import WandbCallback
import wandb

2022-06-16 22:40:01.770265: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
tf.__version__

'2.4.1'

In [4]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [10, 20, 50]
        },
        'dropout': {
            'values': [0.2, 0.5, 0.7]
        },
        'dense': {
            'values': [32, 64, 256, 1024]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
        'optimizer': {
            'values': ['adam', 'nadam', 'sgd', 'rmsprop']
        },
        'activation': {
            'values': ['relu', 'elu', 'selu', 'softmax']
        }
    }
}

In [5]:
IMAGE_SIZE = 320
MODEL_IMAGE_SIZE = 224

In [6]:
sweep_id = wandb.sweep(sweep_config, project='smartpot')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: ioiz8n8v
Sweep URL: https://wandb.ai/hkleee/smartpot/sweeps/ioiz8n8v


In [7]:
df = pd.read_csv('/home/lab38/Multi_proj_6/data/strawberry_smallsizeimg.csv')
# strawberry_smallsizeimg.csv
df.head()

,image,disease,area,grow,points,original,disease-grow
0,/home/lab38/딸기/V006_77_0_00_04_01_13_0_b02_202...,0,1,5,"[{'xtl': 1003, 'ytl': 1167, 'xbr': 2255, 'ybr'...",NaN,0-5
1,/home/lab38/딸기/V006_77_0_00_04_01_13_0_b02_202...,0,1,5,"[{'xtl': 1003, 'ytl': 1217, 'xbr': 2503, 'ybr'...",NaN,0-5
2,/home/lab38/딸기/V006_77_0_00_04_01_13_0_b02_202...,0,1,5,"[{'xtl': 370, 'ytl': 884, 'xbr': 1994, 'ybr': ...",NaN,0-5
3,/home/lab38/딸기/V006_77_0_00_04_01_13_0_b02_202...,0,1,5,"[{'xtl': 739, 'ytl': 1184, 'xbr': 2176, 'ybr':...",NaN,0-5
4,/home/lab38/딸기/V006_77_0_00_04_01_13_0_b02_202...,0,1,5,"[{'xtl': 1031, 'ytl': 1271, 'xbr': 2425, 'ybr'...",NaN,0-5


In [8]:
# label encoding
disease_encoder = LabelEncoder()
disease_encoder.fit(df['disease'])
df['disease'] = disease_encoder.transform(df['disease'])
print(df['disease'].unique())

grow_encoder = LabelEncoder()
grow_encoder.fit(df['grow'])
df['grow'] = grow_encoder.transform(df['grow'])
print(df['grow'].unique())

[0 1 2]
[4 3 2 1 0]


In [9]:
shuffled_df = df.sample(frac=1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df['image'],
                                                   df['disease-grow'],
                                                   stratify=df['disease-grow'],
                                                   test_size=0.2)

In [11]:
train_df = df[df['image'].isin(X_train)]
test_df = df[df['image'].isin(X_test)]

In [12]:
print(train_df['disease'].unique())
print(train_df['grow'].unique())
print(test_df['disease'].unique())
print(test_df['grow'].unique())

[0 1 2]
[4 3 2 1 0]
[0 1 2]
[4 3 2 1 0]


In [13]:
train_gen = ImageDataGenerator(rescale=1./255,
                            rotation_range=20, # 최대 20도까지 회전
                            width_shift_range=0.1, # 최대 x 범위안에서 좌우/상하 이동
                            height_shift_range=0.1,
                            zoom_range=0.2, # 확대 축소 비율,
                            horizontal_flip=True, # 좌우반전
                            vertical_flip=True, # 상하반전
                            fill_mode='nearest')
valid_gen = ImageDataGenerator(rescale= 1. /255.)

In [14]:
def train() :
    tf.keras.backend.clear_session()
    config_defaults = {
        'epochs' : 2,
        'batch_size': 10,
        'learning_rate' : 1e-3,
        'activation': 'relu',
        'optimizer': 'adam'
    }
    
    wandb.init(config=config_defaults)
    config = wandb.config
    
    train_generator = train_gen.flow_from_dataframe(train_df, 
                                               x_col='image',
                                               y_col=['disease', 'grow'],
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               class_mode='multi_output',
                                               batch_size=config.batch_size)
    valid_generator = valid_gen.flow_from_dataframe(test_df,
                                               x_col='image',
                                               y_col=['disease','grow'],
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               class_mode='multi_output',
                                               batch_size=config.batch_size)
    base_model= resnet50.ResNet50(
          weights='imagenet',
          include_top=False,
         )


    base_model.trainable = False 
    
    # resizing model
    input_data = layers.Input((IMAGE_SIZE, IMAGE_SIZE, 3))
    x = tf.keras.layers.experimental.preprocessing.Resizing(MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE)(input_data)
    resizing = Model(inputs=input_data, outputs=x, name='resize')
   
    # model
    inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    x = resizing(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(config.dropout)(x)
    backbone_out = layers.Dense(config.dense, activation=config.activation)(x)

    disease_outputs = layers.Dense(df['disease'].nunique(), activation='softmax',
                            name = 'diease_outputs')(backbone_out)
    grow_outputs = layers.Dense(df['grow'].nunique(), activation='softmax',
                        name = 'grow_outputs')(backbone_out)

    model = Model(inputs=inputs, 
                  outputs=[disease_outputs, grow_outputs],
                  name='strawberry')    
    if config.optimizer=='sgd':
        optimizer = SGD(learning_rate=config.learning_rate)
    elif config.optimizer=='rmsprop':
        optimizer = RMSprop(learning_rate=config.learning_rate)
    elif config.optimizer=='adam':
        optimizer = Adam(learning_rate=config.learning_rate)
    elif config.optimizer=='nadam':
        optimizer = Nadam(learning_rate=config.learning_rate)

    model.compile(loss={
                  'diease_outputs' : 'sparse_categorical_crossentropy',
                  'grow_outputs' : 'sparse_categorical_crossentropy'
                  },
                  optimizer=optimizer,
                  metrics=['accuracy'])    
    
    _ = model.fit(train_generator,
          validation_data=valid_generator,
          verbose=1,
          epochs=config.epochs,
          callbacks=[WandbCallback()],
          steps_per_epoch=len(train_df)//config.batch_size)


In [15]:
# fp11xljb

In [18]:
wandb.agent('fp11xljb', train)

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: rsx3wom1 with config:
wandb: 	activation: softmax
wandb: 	batch_size: 50
wandb: 	dense: 1024
wandb: 	dropout: 0.2
wandb: 	learning_rate: 1e-05
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13291 validated image filenames.
Found 3323 validated image filenames.


2022-06-16 22:50:21.623944: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.00MiB (rounded to 8388608)requested by op RandomUniform
Current allocation summary follows.
2022-06-16 22:50:21.624040: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2022-06-16 22:50:21.624060: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 313, Chunks in use: 313. 78.2KiB allocated for chunks. 78.2KiB in use in bin. 48.5KiB client-requested in use in bin.
2022-06-16 22:50:21.624068: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 206, Chunks in use: 206. 104.0KiB allocated for chunks. 104.0KiB in use in bin. 103.0KiB client-requested in use in bin.
2022-06-16 22:50:21.624074: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 406, Chunks in use: 406. 411.5KiB allocated for chunks. 411.5KiB in use in bin. 40

2022-06-16 22:50:22.378653: W tensorflow/core/framework/op_kernel.cc:1751] Invalid argument: ValueError: callback pyfunc_11 is not found
Traceback (most recent call last):

  File "/home/ubuntu/anaconda3/envs/smartpot/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_11 is not found


2022-06-16 22:50:22.378752: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: Invalid argument: ValueError: callback pyfunc_11 is not found
Traceback (most recent call last):

  File "/home/ubuntu/anaconda3/envs/smartpot/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_11 is not found


	 [[{{node PyFunc}}]]
2022-06-16 22:50:22.453163: W tensorflow/core/framework/op_kernel.cc:1751] Invalid argu

Run rsx3wom1 errored: ResourceExhaustedError()
wandb: ERROR Run rsx3wom1 errored: ResourceExhaustedError()
wandb: Agent Starting Run: e135jlsz with config:
wandb: 	activation: softmax
wandb: 	batch_size: 50
wandb: 	dense: 256
wandb: 	dropout: 0.7
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13291 validated image filenames.
Found 3323 validated image filenames.


wandb: Ctrl + C detected. Stopping sweep.
